In [2]:
%load_ext autoreload
from google.auth import default
credentials, project = default()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
%autoreload 2
from dotenv import load_dotenv
import sys
import os
load_dotenv()
# Add the parent directory of 'src' to the Python path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

    
from src.convert import clean_filename
from src.gcs_storage import upload_to_gcs, get_phrase_path, get_story_names,read_from_gcs, get_phrase_to_story_index_path, get_story_collection_path, get_story_dialogue_path
from src.nlp import plot_vocabulary_growth
from collections import defaultdict
from src.gcs_storage import get_story_index_path
from src.nlp import create_story_index
from src.config_loader import config
COLLECTION = "WarmUp150"


## Sorting stories

We need to now get stories from the story collection path and refer to the phrase.json and index.json files in order to help sort them

In [7]:
all_story_names = get_story_names(collection=COLLECTION)

In [8]:
# let's get all story dialogue


all_story_dialogue = defaultdict(dict)
for story_name in all_story_names:
    story_dialogue = read_from_gcs(config.GCS_PRIVATE_BUCKET, get_story_dialogue_path(story_name, collection=COLLECTION))
    all_story_dialogue[story_name] = story_dialogue

print(all_story_dialogue.keys())


story_index = create_story_index(all_story_dialogue)


upload_to_gcs(story_index, config.GCS_PRIVATE_BUCKET, get_story_index_path(collection=COLLECTION))


dict_keys(['story_airport_wedding_chaos', 'story_birthday_train_adventure', 'story_concert_plans_unravel', 'story_midnight_bakery', 'story_mysterious_wedding_gift', 'story_sunset_adventure_documentary', 'story_the_wedding_planner', 'story_underwater_community_centre'])


Indexing stories...: 100%|██████████| 8/8 [02:43<00:00, 20.45s/it]


'gs://audio-language-trainer-private-content/collections/WarmUp150/story_index.json'

In [9]:
from src.gcs_storage import get_phrase_index_path, get_story_index_path

phrase_index = read_from_gcs(config.GCS_PRIVATE_BUCKET, get_phrase_index_path(collection=COLLECTION))
story_index = read_from_gcs(config.GCS_PRIVATE_BUCKET, get_story_index_path(collection=COLLECTION))



In [12]:
from src.nlp import assign_phrases_to_stories
assignments = assign_phrases_to_stories(
    phrase_index=phrase_index,
    story_index=story_index,
    max_phrases_per_story=50,
)


Optimized Story Sequence:
------------------------------------------------------------
Story                          Score      Verbs      Vocab     
------------------------------------------------------------
story_mysterious_wedding_gift  113        25         63        
story_airport_wedding_chaos    127        35         57        
story_birthday_train_adventure 134        33         68        
story_underwater_community_centre 135        26         83        
story_concert_plans_unravel    140        34         72        
story_sunset_adventure_documentary 148        36         76        
story_midnight_bakery          174        38         98        
story_the_wedding_planner      204        39         126       

Processing story: story_mysterious_wedding_gift
Story has 25 new verbs and 63 new vocabulary words to learn
Target number of phrases for this story: 44
Assigned 44 phrases to story_mysterious_wedding_gift
Story verb coverage from this story's flashcards: 88.0%
Story 

In [13]:
assignments

{'story_mysterious_wedding_gift': [{'phrase': "I can't figure it out",
   'score': 24.0,
   'new_story_verbs': 2,
   'new_story_vocab': 4,
   'new_global_verbs': 2,
   'new_global_vocab': 4,
   'total_new_words': 6},
  {'phrase': 'Do you know the way?',
   'score': 20.0,
   'new_story_verbs': 2,
   'new_story_vocab': 2,
   'new_global_verbs': 2,
   'new_global_vocab': 3,
   'total_new_words': 5},
  {'phrase': "Let's have tea this afternoon",
   'score': 20.0,
   'new_story_verbs': 2,
   'new_story_vocab': 2,
   'new_global_verbs': 2,
   'new_global_vocab': 4,
   'total_new_words': 6},
  {'phrase': "I'm trying to lose weight",
   'score': 19.0,
   'new_story_verbs': 2,
   'new_story_vocab': 1,
   'new_global_verbs': 3,
   'new_global_vocab': 2,
   'total_new_words': 5},
  {'phrase': 'I might get a new job',
   'score': 18.0,
   'new_story_verbs': 2,
   'new_story_vocab': 1,
   'new_global_verbs': 2,
   'new_global_vocab': 3,
   'total_new_words': 5},
  {'phrase': 'Look at that beautiful

In [14]:
from src.nlp import plot_vocabulary_growth_from_assignments
plot_vocabulary_growth_from_assignments(assignments)

In [15]:
from src.gcs_storage import get_story_collection_path
upload_to_gcs(obj = assignments,
            bucket_name=config.GCS_PRIVATE_BUCKET,
            file_name = get_story_collection_path(collection=COLLECTION))

'gs://audio-language-trainer-private-content/collections/WarmUp150/WarmUp150.json'